In [1]:
class Foo:
    def __getitem__(self, item):
        return range(0, 30, 10)[item]

foo = Foo()
print(foo[1])
for x in foo:
    print(x)
    print(20 in foo)


10
0
True
10
True
20
True


In [3]:
import random

class FrenchDeck:

    ranks = [str(x) for x in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()  # 花色

    def __init__(self):
        self._cards = [(rank, suit) for rank in self.ranks for suit in self.suits]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, item):
        return self._cards[item]

    def __repr__(self):
        return str(self._cards)


cards = FrenchDeck()


def set_card(deck, position, value):
    """猴子补丁，实现__setitem__"""
    deck._cards[position] = value  # 需要知道有个_cards属性


FrenchDeck.__setitem__ = set_card  # 打到类对象上，而不是实例
# random.shuffle(cards)

print(cards)



[('2', 'spades'), ('2', 'diamonds'), ('2', 'clubs'), ('2', 'hearts'), ('3', 'spades'), ('3', 'diamonds'), ('3', 'clubs'), ('3', 'hearts'), ('4', 'spades'), ('4', 'diamonds'), ('4', 'clubs'), ('4', 'hearts'), ('5', 'spades'), ('5', 'diamonds'), ('5', 'clubs'), ('5', 'hearts'), ('6', 'spades'), ('6', 'diamonds'), ('6', 'clubs'), ('6', 'hearts'), ('7', 'spades'), ('7', 'diamonds'), ('7', 'clubs'), ('7', 'hearts'), ('8', 'spades'), ('8', 'diamonds'), ('8', 'clubs'), ('8', 'hearts'), ('9', 'spades'), ('9', 'diamonds'), ('9', 'clubs'), ('9', 'hearts'), ('10', 'spades'), ('10', 'diamonds'), ('10', 'clubs'), ('10', 'hearts'), ('J', 'spades'), ('J', 'diamonds'), ('J', 'clubs'), ('J', 'hearts'), ('Q', 'spades'), ('Q', 'diamonds'), ('Q', 'clubs'), ('Q', 'hearts'), ('K', 'spades'), ('K', 'diamonds'), ('K', 'clubs'), ('K', 'hearts'), ('A', 'spades'), ('A', 'diamonds'), ('A', 'clubs'), ('A', 'hearts')]


In [ ]:

import collections

Card = collections.namedtuple('Card', 'rank suit')


class FrenchDeck(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for rank in self.ranks for suit in self.suits]

    def __getitem__(self, item):
        return self._cards[item]

    def __len__(self):
        return len(self._cards)

    def __setitem__(self, key, value):
        self._cards[key] = value

    def __delitem__(self, key):
        del self._cards[key]

    def insert(self, index, value):
        self._cards.insert(index, value)



In [8]:
import abc

class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_cls_method(cls, x):
        pass

In [9]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""

    @abc.abstractmethod
    def pick(self):
        """随机删除元素，然后将其返回
        如果实例为空，这个方法抛出异常LookUpError"""

    def loaded(self):
        """如果至少有一个元素，返回True，否则返回FALSE"""
        bool(self.inspect())

    def inspect(self):
        """返回一个有序元组，由当前元素构成。"""
        items = []
        while True:
            try:
                items.append(self.pick())  # 我们不知道子类如何存储元素，不过为了得到所有元素，只能不断的pick出所有的
            except LookupError:
                break
        self.load(items)  # 刚才的循环后，容器已空，再次加进去，虽然顺序已经和之前不同了
        return tuple(sorted(items))


In [10]:

class Fake(Tombola):
    def pick(self):
        return 12

print(Fake)
fake = Fake()

<class '__main__.Fake'>


TypeError: Can't instantiate abstract class Fake with abstract method load

In [13]:

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, iterable):
        self._items.extend(iterable)
        self._randomizer.shuffle(self._items)  # 类似于random.shuffle()就地随机打乱顺序

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('BingoCage is empty!')

    def __call__(self, *args, **kwargs):
        return self.pick()

bb = BingoCage([1,2,3])
bb

In [14]:
class LotteryBlower(Tombola):
    def __init__(self, items):
        self._items = list(items)

    def load(self, iterable):
        self._items.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._items))  # 随机位置
        except ValueError:
            raise LookupError('LotteryBlower is empty!')
        self._items.pop(position)

    def loaded(self):
        return bool(self._items)

    def inspect(self):
        return tuple(sorted(self._items))

In [18]:
@Tombola.register  # 注册为Tombola的虚拟子类
class TomboList(list):
    def pick(self):
        if self:
            position = random.randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('TomboList is empty!')

    load = list.extend

    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(sorted(self))

print(issubclass(TomboList, Tombola))
t = TomboList()
print(isinstance(t, Tombola))

print(TomboList.__mro__)

print(Tombola.__subclasses__())

print(list(Tombola._re))


True
True
(<class '__main__.TomboList'>, <class 'list'>, <class 'object'>)
[<class '__main__.Fake'>, <class '__main__.BingoCage'>, <class '__main__.BingoCage'>, <class '__main__.LotteryBlower'>]


AttributeError: type object 'Tombola' has no attribute '_abc_registry'

In [19]:
class Str:
    def __len__(self):
        return 23

from collections import abc

print(issubclass(Str, abc.Sized))
print(isinstance(Str(), abc.Sized))

True
True


In [20]:
import abc

class A(abc.ABC):
    pass

class B:
    pass

print(issubclass(B, A))
A.register(B)
print(issubclass(B, A))

False
True


In [21]:
from collections import abc
print(issubclass(tuple, abc.Sequence))

from collections import abc
print(issubclass(tuple, abc.Sequence))

True
True


In [22]:
import abc

class Good(abc.ABC):
    @abc.abstractmethod
    def price(self):
        pass

class Food(Good):
    def price(self):
        print("{} price:$4".format(__class__.__name__))

class Clothes(Good):
    def price(self):
        print("{} price:$5".format(__class__.__name__))

In [23]:
f = Food()
C = Clothes()

f.price()
C.price()

Food price:$4
Clothes price:$5


In [44]:
import abc

class Good(abc.ABC):

    @abc.abstractmethod
    def pp(self):
        pass

    @abc.abstractmethod
    def dd(self):
        pass

@Good.register  # 注册为Tombola的虚拟子类
class T():

    def pp(self):
        print("1")


# Good.register(T)
a = T()
a.pp()

print(isinstance(a, Good))
print(issubclass(T, Good))

1
True
True


In [57]:
class A():
    a = 1

    def __init__(self, x):
        self.x = x

class B(A):
    b = 2

bb = B(1111)
bb.b = 200
print(bb.b, bb.a, bb.x, B.b, bb.b)
print(bb.b, bb.a, bb.x)

200 1 1111 2 200
200 1 1111
